# **CapStone Project - The Battle of neighbourhoods (Week1)**

## Table of Contents

* [Introduction: Business Problem](#Introduction)
* [Data](#Data)

## 1. Introduction

In this project we will try to find an optimal resort in a tourist place named *Kodaikanal, TamilNadu* in India.

**Kodaikanal** is a hill town in the southern Indian state of Tamil Nadu. It’s set in an area of granite cliffs, forested valleys, lakes, waterfalls and grassy hills making it a tourister choice with its sceneric beauty. Since there are lots of resorts in Kodaikanal, we will try to detect locations that are optimal for travellers with various interests. Some tourists may like resorts closer to the town, some may like resorts closer to tourist spots and so on. We will provide the details of each location so the tourist can choose the resort of their choice.

## 2. Data

The factors that will influence our decision are:

 - Venues Close by the resort
 
 - Ratings on the Resort
 
 - Likes on the Resort

Following data sources will be needed to extract/generate the required information:

 - Resorts in kodaikanal are obtained using Foursquare API

 - The details of the resort like the Rating,Number of Likes are obtained using Foursquare premium API

 - Number of restaurants,tourist spots and their type and location near every Resort around 2.5 km radius will be obtained using Foursquare API


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
address = 'Kodaikanal,Tamil Nadu'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kodaikanal City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kodaikanal City are 10.273275349999999, 77.51160822153315.


In [3]:
CLIENT_ID = 'R51UD0XCGDUSCMGV2AMLFHWZIJRBWWVAB0SELQWQ41QK5Z4Y' # your Foursquare ID
CLIENT_SECRET = 'JNOQ05S2XLH02TYZBXV3IL3DANX5CCJPSU41ZSSGHLN4NELV' # your Foursquare Secret
VERSION = '20210107' #fourquare API version
LIMIT = 100 # A default Foursquare API limit value
radius=30000

The category id for Hotel/resort is '4bf58dd8d48988d1fa931735' from Foursquare API category list and we are passing this category id to get only the Resorts in Kodaikanal within 30 km radius

In [4]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT,
            '4bf58dd8d48988d1fa931735')
                       
        # make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']
        

In [5]:
venues_list=[]
venues_list.append([(
            v['venue']['name'], 
            v['venue']['id'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

df_resorts = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
df_resorts.columns = ['Name','Venue ID','Latitude','Longitude','Category']

In [6]:
df_resorts

,Name,Venue ID,Latitude,Longitude,Category
0,Carlton Hotel,4c1601317f7f2d7f743be368,10.234211,77.488522,Hotel
1,The Kodaikanal Club,4c34be637cc0c9b6f9d9f39a,10.234079,77.491351,Resort
2,Hotel Kodai International,4d59fbd17e22370438f5b773,10.242568,77.498038,Hotel
3,Apple Valley Resort,50a20b69e4b02fa5344ce28c,10.236985,77.495837,Hotel
4,Hotel Cliffton,4d58dcde7e2237043617aa73,10.238823,77.484139,Hotel
5,Villa Retreat,4c164afda9c220a108605a9d,10.232191,77.494358,Hotel
6,Hotel Vel's Court,4ef095ae77c8053fbd0fc24a,10.441988,77.516294,Hotel
7,Sterling valley view resort,4dca3084b0fb9c8f8ae0172d,10.268499,77.490931,Resort
8,"Sterling Holidays, Kodai - Valley View",51b87edc7dd2c2c158dd08ee,10.268134,77.488890,Resort
9,GRT Nature Trails. Kodaikanal,5621aaa7498e25669673fd3e,10.275460,77.486388,Resort


In [7]:
# create map
map_resorts = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to the map

for lat, lon, poi in zip(df_resorts['Latitude'], df_resorts['Longitude'], df_resorts['Name']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.Marker(
        [lat, lon],
        popup=label).add_to(map_resorts)

map_resorts